# Intro to Dijkstra's shortest path algorithm

This notebook serves as a tutorial describing how Djikstra's shortest path algorithm works, as well as providing a Python implementation with extensive comments. The code was re-written based on Alex Woods's implementation [here](https://alexhwoods.com/dijkstra/). I also recommend the visualizations [here](https://brilliant.org/wiki/dijkstras-short-path-finder/) for seeing how the algorithm proceeds in a step-by-step fashion.  

The following code implements Dijkstra's route-finding algorithm to find the shortest paths through a directed graph. The code has three components:
1. A `Graph` class that generates the data structure. This class contains _vertices_ (also called _nodes_) that represent intersection points, and _edges_, which correspond to paths between vertices. These edges have _weights_, or numeric values representing the distance between vertices along that edge.
2. A `find_paths` function which measures the distances between vertices, finding the shortest possible paths.
3. A `shortest_path` function which takes two vertex IDs as inputs and returns the shortest path

We'll start out by implementing the `Graph` class, which is explained above.

In [14]:
import collections
import math

# Define the Graph class. it has vertices, directed edges, and weights for the edges.
class Graph(object):
    """A class of vertices and edges connecting them"""
    
    def __init__(self):
        self.vertices = set()  # using set instead of list avoids duplicates
        self.edges = collections.defaultdict(list)  # empty edge dict with list-type values
        self.weights = {}
        
    def add_vertex(self, vertex_id):
        """Add a new vertex identified as v"""
        self.vertices.add(vertex_id)
    
    def add_edge(self, start_vertex, end_vertex, weight):
        """Add an edge connecting start_vertex and end_vertex with weight"""
        if start_vertex == end_vertex:
            pass  # no cycles allowed
        self.edges[start_vertex].append(end_vertex)
        self.weights[(start_vertex, end_vertex)] = weight
    
    def __str__(self):
        string = "Vertices: " + str(self.vertices) + "\n"
        string += "Edges: " + str(self.edges) + "\n"
        string += "Weights: " + str(self.weights)
        return string

Next, we'll implement the `dijkstra` function. This function takes two arguments:
- `graph`, an object of class `Graph` containing all of the vertices, edges, and weights.
- `start_vertex`, the starting point that you want to measure distances to.  

The function proceeds through the following steps:
1. Initialize `distances`, a dictionary that will hold the minimum distance from `start_vertex`. `distances` is initialized with all values being infinity, except for `distances[start_vertex]`, which is initialized as zero.
2. Initialize `previous`, a dictionary which will hold the previous vertex along the shortest path from `start_vertex` to every other vertex in `graph`.
3. Initialize `already_visited`, which will hold the vertices that have been traversed by the `djikstra` function to prevent looping.
4. Iterate through all of the vertices in `graph` until all have been visited, and do the following:  

    4a. pick the vertex `v` with the lowest value for `distances` that isn't in `already_visited`. This will be `start_vertex` in the first pass (the only one with value below infinity in `distances`); the `distances` values for other vertices are updated within the loop.  
    4b. get the distance from `start_vertex` to `v` from `distances`.  
    4c. get the list of `v`'s neighbors from `graph.edges[v]`, which stores edges as lists of connected vertices.  
    4d. for each neighbor vertex that isn't in `already_visited`, check to see if traveling to the neighbor through `v` is shorter than the currently known shortest path (stored in `distances`). If it is, replace `distances[neighbor]` with `distances[v] + graph.weights[v, neighbor]`.  
    4e. if the shortest path was through `v`, update `previous[neighbor]` to reflect that.  
    4f. having checked all neighbors for vertex `v`, add `v` to `already_visited`.  
5. Return `distances`, which indicates the minimum distance to each vertex, and `previous`, which stores the previous vertex along the shortest path from `start_vertex` to each vertex in `graph`.

In [26]:
# define the Dijkstra function, which finds the shortest distances between vertices
# and the previous vertex along the shortest route to each vertex
def dijkstra(graph, start_vertex):
    """Find the shortest distance to each vertex beginning at start_vertex"""
    # initialize distance to each vertex in graph from start_vertex. This should be
    # initialized as 0 for start_vertex and infinity for all other vertices.
    distances = dict.fromkeys(list(graph.vertices), math.inf)  # Step 1
    distances[start_vertex] = 0
    # initialize a dictionary called previous which will indicate which vertex precedes
    # each non-start vertex in the shortest path
    previous = dict.fromkeys(list(graph.vertices), None)  # Step 2
    # generate a set of tested vertices to track where you've been
    already_visited = set()  # Step 3
    # iterate over vertices with shortest distances.
    while already_visited != graph.vertices:  # Step 4
        # get the un-visited vertex with the shortest distance from start_vertex
        v = min((set(distances.keys()) - already_visited), key=distances.get)  # Step 4a
        v_dist = distances[v]  # Step 4b
        for neighbor in graph.edges[v]:  # Step 4c
            # the if block below comprises Step 4d-e
            if neighbor not in already_visited:
                full_dist_this_path = v_dist + graph.weights[v, neighbor]
                # next, check if this is the shortest path yet to neighbor
                if distances[neighbor] > full_dist_this_path:
                    distances[neighbor] = full_dist_this_path  # end of step 4d
                    previous[neighbor] = v  # step 4e
        already_visited.add(v)  # step 4f
    return distances, previous  # step 5


Finally, we'll implement `shortest_path`. This function takes three arguments: `graph`, `start_vertex`, and `end_vertex`, which correspond to a graph as described above, and the IDs for the beginning and ending vertices that you want a path for, respectively. The function generates the minimum distances and the previous vertices along paths using `djikstra(graph, start_vertex)`, and then generates a list of vertices along the shortest path from `start_vertex` to `end_vertex` by proceeding backward through `previous` from `end_vertex`.

In [27]:
# define shortest_path, which uses previous from dijkstra to get the shortest path
def shortest_path(graph, start_vertex, end_vertex):
    """Find the shortest path from start_vertex to end_vertex"""
    distances, previous = dijkstra(graph, start_vertex)
    path = [end_vertex]
    next_vertex = previous[end_vertex]
    while next_vertex is not None:
        path.append(next_vertex)
        next_vertex = previous[next_vertex]
    path.reverse()  # order from start to end
    return path

## Test case

See below for an implementation of a test case using the code provided above. The graph in the test case is represented in the image below, and the path from vertex __A__ to vertex __H__ is found.  
<img src='IMG_7364.JPG'>

In [25]:
### TEST RUN ###
G = Graph()
G.add_vertex('a')
G.add_vertex('b')
G.add_vertex('c')
G.add_vertex('d')
G.add_vertex('e')
G.add_vertex('f')
G.add_vertex('g')
G.add_vertex('h')
 
G.add_edge('a', 'b', 1)
G.add_edge('a', 'c', 4)
G.add_edge('a', 'd', 3)
G.add_edge('b', 'e', 4)
G.add_edge('c', 'e', 2)
G.add_edge('d', 'f', 1)
G.add_edge('e', 'f', 2)
G.add_edge('e', 'g', 3)
G.add_edge('f', 'h', 1)
G.add_edge('g', 'h', 4)
 
print('Graph: \n{}'.format(G)) 
print()
print('Shortest path from {} to {}: {}'.format('a','h',shortest_path(G, 'a', 'h')))

Graph: 
Vertices: {'a', 'e', 'f', 'd', 'g', 'b', 'h', 'c'}
Edges: defaultdict(<class 'list'>, {'a': ['b', 'c', 'd'], 'b': ['e'], 'c': ['e'], 'd': ['f'], 'e': ['f', 'g'], 'f': ['h'], 'g': ['h']})
Weights: {('a', 'b'): 1, ('a', 'c'): 4, ('a', 'd'): 3, ('b', 'e'): 4, ('c', 'e'): 2, ('d', 'f'): 1, ('e', 'f'): 2, ('e', 'g'): 3, ('f', 'h'): 1, ('g', 'h'): 4}

Shortest path from a to h: ['a', 'd', 'f', 'h']
